# Double DQN
Double DQN [1] improves the update step from DQN [2] by using an idea from Double Q-learning [3].
The target for the update step in DQN uses the max function to find the maximum action value for the next state [2]:

$$ y_t = r_{t+1} + \gamma \cdot \max_{a}Q(s_{t+1}, a, \Theta^{-}_{t})$$

This can lead to an overestimation of action values. Double DQN reduces this problem by using two different neural networks, one for action selection, the second for predicting the q-value corresponding to the action: 

$$ y_t = r_{t+1} + \gamma \cdot Q(s_{t+1}, \text{argmax}_{a} Q(s_{t+1}, a, \Theta_{t}), \Theta^{-}_{t})$$

where $\Theta$ and $\Theta^{-}$ denote the qnet and target net respectively [2].
This helps reduce the overestimation of q-values, which according to [2] can be caused by estimation errors of any kind (including noise).

## Results
I ran the Bsuite experiments [4] and used the analysis Jupyter Notebook to produce some plots for comparison.
The parameters were the same as for the best run of the basic DQN algorithm, with an additional flag that indicates if DDQN is used or not.

<em>
Run013:  
    
qnet_settings = {"layers_sizes": [50], "batch_size": 64}  
    
settings = {"batch_size": qnet_settings["batch_size"], "epsilon_start": 1.0, "epsilon_decay": 0.999,  
            "epsilon_min": 0.025, "gamma": 0.99, "buffer_size": 200000, "lr": 1e-3, "qnet_settings": qnet_settings,  
            "start_optimization": 64, "update_qnet_every": 2, "update_target_every": 50, "ddqn": True}
    </em>

Note that <em>update_target_every</em> was changed from 25 to 50 since I removed a bug that resulted in updates of the target neural network only on when an optimization step was performed.
![Radar plot with a comparison between DQN and DDQN](./figures/ddqn_radar.png)

![Bar plot comparing the differences between DQN and DDQN in the individual experiments.](./figures/ddqn_barplot.png)

The plots above show that with the given parameters, DQN has a slightly better performance than DDQN.
Interestingly, DDQN has worse performance on the noise and scaling experiments. I had expected DDQN to be better in the presence of noise. On the Mountain Car experiments DDQN is slightly better than DQN.

## Discussion

Possible causes for the lack of improvement of DDQN:
- Even though DDQN seems like a small update to the code and the debugger showed reasonable results, there might still be a bug in the code.
- The hyperparameters of the agent were not properly tuned. The authors of [1] had a slightly better performance of DQN on a few of the Atari games. After tuning the parameters of DDQN the performance got another big increase (see Figure 2 of [2]) and showed better performance than DQN on all games.  
- In particular, the frequency of the target network updates might not be chosen well. In [3], the authors update the target network every 10.000 steps, which is infeasible since some of the BSuite experiments only have 10,000 steps. In the [BSuite DQN baseline](https://github.com/deepmind/bsuite/tree/master/bsuite/baselines/dqn), the target network is updated every 4 steps. Updating the target network too frequently would diminish the effect of using it in the first place. Therefore, updating it every 25-50 steps seemed like a good compromise. I now started another run using the same parameters but a 5 times lower update frequency for the target net (i.e. every 250 steps) to see if that improves the agent's performance. 

So far I have not performed a proper parameter search for my agent, since I am running all computations on my laptop's GPU. My current plan is to first implement all the improvements from the Rainbow manuscript [5], then optimize the hyperparameters in GCP and finally perform an ablation study using those optimized parameters. 

## References
[1] Van Hasselt, H.; Guez, A.; Silver, D. Deep reinforcement learning with double q-learning. In: Thirtieth AAAI conference on artificial intelligence. 2016.  
[2] Mnih, Volodymyr, et al. Human-level control through deep reinforcement learning. Nature, 2015.  
[3] Van Hasselt, H. Double Q-learning. In: Advances in neural information processing systems. 2010.  
[4] Osband, Ian, et al. "Behaviour Suite for Reinforcement Learning." arXiv preprint arXiv:1908.03568, 2019.  
[5] Hessel, Matteo, et al. Rainbow: Combining improvements in deep reinforcement learning. In: Thirty-Second AAAI Conference on Artificial Intelligence. 2018.  

The figures here were again produced by the analysis Jupyter Notebook from [the BSuite code repository](https://github.com/deepmind/bsuite).